In [ ]:
'''
该API之中的几个参数需要我们有所了解：

file_path：CSV数据文件的路径；
batch_size：我们要划分数据集合的批次大小；
label_name：我们要进行预测的列；
na_value：该API会将文件中的空白值替换为 na_value ；
num_epochs：重复读取该数据集合的数量，通常设置为 1，因为我们只需要读取一遍数据集即可。
'''
dataset = tf.data.experimental.make_csv_dataset(
      file_path, batch_size, label_name, na_value, num_epochs
    )


In [ ]:
import tensorflow as tf

# 获取数据集
train_path = tf.keras.utils.get_file("train.csv",
            "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
valid_path = tf.keras.utils.get_file("eval.csv",
            "https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

# 通过CSV文件构建数据集
train_data = tf.data.experimental.make_csv_dataset(train_path, batch_size=32,
        label_name='survived', na_value="?", num_epochs=1,
        ignore_errors=True )
valid_data = tf.data.experimental.make_csv_dataset(valid_path, batch_size=32,
        label_name='survived', na_value="?", num_epochs=1,
        ignore_errors=True )

'''
定义离散列并进行处理
对于离散数据，我们进行了以下几步处理：

定义离散列以及其可能的值（我们称作字典）；
使用 tf.feature_column.categorical_column_with_vocabulary_list 这个函数 API 构建 TensorFlow 能够识别的离散列；
将所有的离散列合在一起以待后面使用。
'''
caetogries = {'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']}
categorical_columns = []
for feature, vocab in caetogries.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

'''
定义连续列
对于连续数据，我们同样进行了以下几步处理：

定义连续列；
使用 tf.feature_column.numeric_column 这个函数 API 构建 TensorFlow 能够识别的连续列；
将所有的连续列合在一起以待后面使用。
'''
numerical_names = {'age',
    'n_siblings_spouses',
    'parch',
    'fare'}
numerical_columns = []
for feature in numerical_names:
    num_col = tf.feature_column.numeric_column(feature)
    numerical_columns.append(num_col)


'''
定义模型
最后在模型的第一层，我们添加了一个特征的预处理层，
因为我们输入的特征有的是连续值，有些是离散值，
因此我们需要该层进行特征的预处理。
'''
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(categorical_columns + numerical_columns),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# 模型训练
model.fit(train_data, epochs=20)

# 模型评估
model.evaluate(valid_data)
